In [5]:
import os
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd

dotenv_path = Path("../../.env")
if dotenv_path.exists():
    print("Loaded .env file!")
load_dotenv(str(dotenv_path))

pd.options.display.max_rows = 200
pd.options.display.max_columns = 999

df = pd.read_parquet(Path(os.environ["PROJECT_HOME_DIR"]) / "data/corrected_train_v1.pq").drop(columns=["token_text"])

pred_paths = [
    "/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-empathic-enterprise-819-2/preds.pq"
    # "/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-1024-f3/preds.pq",
    # "/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-1024-f2/preds.pq",
    # "/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-1024-f1/preds.pq",
    # "/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-1024-f0/preds.pq",
    # "/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-long-f0/preds.pq",
    # "/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-long-f1/preds.pq",
    # "/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-long-f2/preds.pq",
    # "/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-long-f3/preds.pq",
]

tdf = pd.concat([pd.read_parquet(p) for p in pred_paths])
df = df.merge(tdf, on=["token", "document"], how="right")

Loaded .env file!


In [6]:
main_cols = ["token_text", "label", 'B-EMAIL', 'B-ID_NUM',
       'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL',
       'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM',
       'I-STREET_ADDRESS', 'I-URL_PERSONAL', 'O',
    "document",
    "token",]

no_bio_cols = [
    "EMAIL",
    "ID_NUM",
    "NAME_STUDENT",
    "PHONE_NUM",
    "STREET_ADDRESS",
    "URL_PERSONAL",
    "USERNAME",
]

main_no_bio = [x for x in main_cols if x.split("-")[-1] not in no_bio_cols] + no_bio_cols


maybe_cols = [
    "I-EMAIL",
    "I-USERNAME",
]

In [7]:
from piidd.processing.post import manual_check


df2 = manual_check(df)

/drive2/kaggle/pii-dd/piidd/processing/post.py:488: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  pred_df["token_text"].str.contains("(http)|(www)|(://)"),


In [8]:
df2

,document,token,label,token_text
0,10,0,B-NAME_STUDENT,Diego
1,10,1,I-NAME_STUDENT,Estrada
2,10,464,B-NAME_STUDENT,Diego
3,10,465,I-NAME_STUDENT,Estrada
4,86,6,B-NAME_STUDENT,Eladio
...,...,...,...,...
873,22206,464,B-NAME_STUDENT,Madelon
874,22206,540,B-NAME_STUDENT,Madelon
875,22206,884,B-NAME_STUDENT,Madelon
876,22406,185,B-NAME_STUDENT,Rita


In [9]:
from piidd.training.utils import pii_fbeta_score_v2

pii_fbeta_score_v2(df2, df.loc[df["label"] != "O"], ".", beta=5, save_csv=True)

0.9863774225055262

In [4]:
c = "NAME_STUDENT"

df[df[c]>0.1].sort_values(c).style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,document,token,label,token_text,EMAIL,ID_NUM,NAME_STUDENT,O,PHONE_NUM,STREET_ADDRESS,URL_PERSONAL,USERNAME,row_id
1057170,21514,312,O,Saeid,0.000198,0.000355,0.100972,0.933404,0.000220,0.000190,0.001000,0.000335,1057170
392137,12330,163,B-NAME_STUDENT,Miriam,0.000390,0.000632,0.100979,0.893976,0.001116,0.000916,0.001331,0.000659,392137
1027690,21142,253,O,Adam,0.000358,0.000197,0.101334,0.896568,0.000310,0.000459,0.000401,0.000372,1027690
1027986,21142,566,O,Adam,0.000312,0.000182,0.103944,0.894192,0.000261,0.000384,0.000306,0.000419,1027986
71462,6718,151,O,Rob,0.000335,0.000256,0.112563,0.885336,0.000438,0.000408,0.000364,0.000301,71462
863051,18870,0,O,LS,0.001317,0.003039,0.113020,0.876496,0.001204,0.001111,0.001471,0.002342,863051
906343,19530,360,O,Susan,0.000462,0.000259,0.113676,0.883557,0.000264,0.000516,0.000366,0.000898,906343
71432,6718,120,O,Rob,0.000254,0.000222,0.113716,0.884352,0.000353,0.000393,0.000389,0.000322,71432
392101,12330,125,B-NAME_STUDENT,Miriam,0.000375,0.000573,0.116829,0.878700,0.000892,0.000815,0.001201,0.000616,392101
298597,11262,30,O,Anna,0.000256,0.000250,0.116984,0.880791,0.000503,0.000497,0.000378,0.000341,298597


In [5]:
c = "URL_PERSONAL"

df[df[c]>0.1].sort_values(c).style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,document,token,label,token_text,EMAIL,ID_NUM,NAME_STUDENT,O,PHONE_NUM,STREET_ADDRESS,URL_PERSONAL,USERNAME,row_id
264842,10802,1062,O,business/#701fa2714d79,0.011679,0.067182,0.008410,0.999779,0.011889,0.004576,0.120625,0.017154,264842
316678,11498,978,O,http://www.imrankhanfoundation.org/about-ikf/the-board/,0.009644,0.009558,0.055732,0.999866,0.009686,0.004982,0.141782,0.014275,316678
17085,2722,1037,O,https://www.solfiten.com/,0.001682,0.000841,0.013773,0.998239,0.002385,0.000970,0.206660,0.002264,17085
369850,12090,328,O,http://www-rohan.sdsu.edu/~digger/305/toulmin_model.htm,0.684237,0.004354,0.006413,0.999633,0.002713,0.002213,0.226835,0.004625,369850
1073528,21722,710,O,https://issuu.com/cpitrade/docs/broadcast_pro_march_2021,0.001822,0.000692,0.003969,0.996762,0.000946,0.000724,0.237207,0.000983,1073528
836582,18514,301,O,https://www.youtube.com/watch?v=HZ4gVlT93R4&t=1s,0.006776,0.026868,0.007429,0.922376,0.020113,0.004061,0.565365,0.024150,836582
135795,9110,137,O,https://www.youtube.com/watch?v=t3kKechcwYM&feature=emb_title,0.003234,0.007168,0.006603,0.902094,0.007964,0.002541,0.628568,0.007088,135795
836723,18514,453,O,https://www.youtube.com/watch?v=2Ev1ciZ5e1g&t=1s,0.007603,0.028192,0.009465,0.902713,0.016981,0.004310,0.633432,0.023365,836723
70601,6694,621,O,https://www.youtube.com/watch?v=5Zuq92PUr5E,0.002444,0.010787,0.007661,0.661505,0.008976,0.002034,0.638036,0.006509,70601
552373,14446,227,O,https://www.youtube.com/watch?v=wouXuqJiCro,0.002182,0.011975,0.005733,0.679046,0.013095,0.002205,0.659318,0.007501,552373


In [14]:
df["max_pred"] = df[no_bio_cols + ["O"]].idxmax(axis=1).values

In [20]:
df[df["max_pred"] != "O"].iloc[200:400, :]

,document,token,label,token_text,EMAIL,ID_NUM,NAME_STUDENT,O,PHONE_NUM,STREET_ADDRESS,URL_PERSONAL,USERNAME,row_id,max_pred
46272,4842,6,I-NAME_STUDENT,Peters,0.000219,0.000056,0.998226,0.001173,0.000051,0.000085,0.000110,0.000079,46272,NAME_STUDENT
47134,4878,5,B-NAME_STUDENT,Juan,0.000167,0.000076,0.997800,0.001656,0.000039,0.000081,0.000083,0.000098,47134,NAME_STUDENT
47135,4878,6,I-NAME_STUDENT,Jose,0.000215,0.000082,0.997667,0.001591,0.000048,0.000123,0.000133,0.000141,47135,NAME_STUDENT
47136,4878,7,I-NAME_STUDENT,Diaz,0.000195,0.000093,0.998077,0.001217,0.000050,0.000129,0.000105,0.000134,47136,NAME_STUDENT
47623,4998,4,B-NAME_STUDENT,Ernesto,0.000243,0.000109,0.996789,0.002316,0.000108,0.000103,0.000168,0.000165,47623,NAME_STUDENT
47624,4998,5,I-NAME_STUDENT,Young,0.000260,0.000066,0.997980,0.001336,0.000045,0.000101,0.000092,0.000120,47624,NAME_STUDENT
48433,5054,6,B-NAME_STUDENT,Carlos,0.000081,0.000032,0.995877,0.003787,0.000039,0.000061,0.000071,0.000052,48433,NAME_STUDENT
48434,5054,7,I-NAME_STUDENT,Pablo,0.000099,0.000038,0.996545,0.003072,0.000036,0.000057,0.000076,0.000077,48434,NAME_STUDENT
50011,5258,802,B-NAME_STUDENT,Nitish,0.000116,0.000053,0.996635,0.002727,0.000064,0.000052,0.000147,0.000205,50011,NAME_STUDENT
50012,5258,803,I-NAME_STUDENT,Kumar,0.000200,0.000061,0.995735,0.003471,0.000056,0.000058,0.000182,0.000236,50012,NAME_STUDENT


In [5]:
df[df["B-EMAIL"] > 0.1][main_cols].style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,O,document,token
79941,john03@yahoo.com,B-EMAIL,0.998147,0.000126,0.000164,0.003237,0.000165,0.000334,0.002352,0.000270,0.000123,0.000106,0.000106,0.000139,0.001582,6900,12
80358,john03@yahoo.com,B-EMAIL,0.997443,0.000153,0.000185,0.003890,0.000185,0.000508,0.003079,0.000359,0.000170,0.000134,0.000151,0.000180,0.002574,6900,469
86127,lisarose@gmail.com,B-EMAIL,0.997279,0.000104,0.000142,0.001004,0.000348,0.000817,0.004100,0.000647,0.000581,0.000303,0.000672,0.000271,0.003650,7308,3
86128,diazkristen@gmail.com,B-EMAIL,0.998819,0.000056,0.000086,0.000722,0.000183,0.000405,0.002106,0.000375,0.000376,0.000279,0.000444,0.000265,0.002906,7308,5
86481,lisarose@gmail.com,B-EMAIL,0.996619,0.000103,0.000160,0.001098,0.000393,0.000909,0.005246,0.000635,0.000615,0.000312,0.000637,0.000308,0.004003,7308,380
86482,diazkristen@gmail.com,B-EMAIL,0.998466,0.000070,0.000092,0.000848,0.000261,0.000447,0.003465,0.000348,0.000371,0.000233,0.000386,0.000264,0.002550,7308,382
103853,andrew58@gmail.com,B-EMAIL,0.996685,0.000173,0.000360,0.001120,0.000150,0.000818,0.000648,0.000314,0.000250,0.000097,0.000116,0.000279,0.005371,8344,14
492235,gdelossantos,O,0.246162,0.003808,0.030542,0.016761,0.004336,0.006262,0.824869,0.010041,0.038210,0.004118,0.007111,0.006205,0.120465,13144,431
577316,hbrown@yahoo.com,B-EMAIL,0.992086,0.000240,0.001176,0.003903,0.000302,0.000801,0.009108,0.000613,0.001181,0.000385,0.000336,0.000419,0.009887,14484,294
1176416,matthew72@hotmail.com,B-EMAIL,0.998699,0.000273,0.000347,0.000162,0.000087,0.000539,0.001380,0.000123,0.000111,0.000191,0.000135,0.000137,0.001709,2769,22


In [4]:
df[df["B-USERNAME"] > 0.1][main_cols].sort_values("B-USERNAME").style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,O,document,token
4434440,:,O,0.001791,0.003834,0.007713,0.001524,0.003913,0.009894,0.100510,0.005357,0.005810,0.003169,0.009981,0.001678,0.844826,20531,18
2338627,1/2,O,0.005119,0.013323,0.124515,0.026251,0.021735,0.005237,0.100568,0.011205,0.530609,0.005458,0.106504,0.009947,0.393426,1290,558
4434459,p3,O,0.003585,0.002589,0.015153,0.001158,0.005645,0.002305,0.103011,0.001333,0.004342,0.001998,0.002604,0.002642,0.956465,20531,39
1555432,LG,O,0.009787,0.003361,0.023197,0.005193,0.006186,0.003420,0.111547,0.012952,0.173956,0.004925,0.016678,0.002011,0.626788,11901,696
1200693,Qais,B-NAME_STUDENT,0.000609,0.007925,0.928739,0.001573,0.001670,0.001126,0.111832,0.001459,0.051991,0.000579,0.000766,0.001157,0.016164,4717,11
1183162,Yuniarno,O,0.011552,0.006432,0.049712,0.014941,0.012630,0.011954,0.114589,0.012172,0.352952,0.011527,0.119163,0.016003,0.568509,3709,2372
4056896,karnege,O,0.001002,0.002740,0.125811,0.001082,0.003378,0.015903,0.122332,0.001821,0.020343,0.001225,0.006573,0.001632,0.791931,14859,155
1250188,vfe&ved=0CAIQjRxqFwoTCODK7paehu0CFQAAAAAdAAAAABAJ,O,0.032948,0.996894,0.007387,0.028020,0.024640,0.040331,0.122491,0.522480,0.007899,0.009466,0.031500,0.009664,0.995270,7137,1381
1817447,Erfjkglfstxhhjkhllhgtdaavhhjhikugteawdjjkhjgyjtfge,O,0.023115,0.934422,0.148655,0.028994,0.021060,0.018690,0.125115,0.128346,0.071313,0.020958,0.049351,0.030077,0.433901,15461,367
2338669,2/2,O,0.005818,0.010147,0.127018,0.022087,0.036423,0.003491,0.129425,0.008245,0.550670,0.005711,0.119529,0.012435,0.334714,1290,603


In [7]:
df[df["I-USERNAME"] > 0.1][main_cols].style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

KeyError: 'I-USERNAME'

In [5]:
df[df["B-STREET_ADDRESS"] > 0.05][main_cols].sort_values("B-STREET_ADDRESS").style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,O,document,token
1183153,Arik,O,0.018255,0.014075,0.571408,0.007400,0.054631,0.007931,0.177857,0.011195,0.072781,0.006788,0.036886,0.014131,0.303312,3709,2362
1264062,Quentin,B-NAME_STUDENT,0.003063,0.004836,0.692602,0.009536,0.056212,0.003662,0.027880,0.004704,0.061876,0.002098,0.008886,0.004292,0.120355,7993,0
2429242,Basavaraju,B-NAME_STUDENT,0.006402,0.011261,0.268397,0.002928,0.059220,0.012906,0.013943,0.006344,0.156229,0.004010,0.034691,0.007628,0.592106,8642,0
1799645,Niñez,O,0.004074,0.004637,0.017443,0.003935,0.065733,0.004044,0.004258,0.004278,0.006233,0.001872,0.016550,0.002653,0.931377,15145,14
1183156,Cucun,O,0.009535,0.012024,0.113139,0.008577,0.087824,0.010240,0.090819,0.008161,0.057355,0.006026,0.064785,0.028613,0.688079,3709,2365
3778959,UVA,O,0.003530,0.302732,0.361103,0.100934,0.144408,0.005328,0.016373,0.011449,0.014023,0.006440,0.010021,0.010505,0.013154,11387,495
1163681,Tino,B-NAME_STUDENT,0.018522,0.016728,0.249459,0.007856,0.149398,0.010802,0.064367,0.014624,0.047015,0.017364,0.038548,0.010326,0.354990,609,92
3779271,UVA,O,0.002814,0.198067,0.402393,0.147991,0.160116,0.005090,0.021656,0.011009,0.012816,0.004746,0.006139,0.010035,0.017127,11387,826
3780006,UVA,O,0.003615,0.044643,0.579746,0.073724,0.168649,0.006494,0.020960,0.007306,0.019048,0.003660,0.005415,0.012922,0.053818,11387,1602
1181674,Cucun,O,0.003520,0.018732,0.061357,0.004402,0.290030,0.005262,0.027703,0.004427,0.018121,0.001697,0.025512,0.007476,0.806455,3709,644


In [6]:
df[df["I-STREET_ADDRESS"] > 0.05][main_cols].sort_values("I-STREET_ADDRESS").style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,O,document,token
3849671,-2,O,0.004007,0.008112,0.009867,0.002632,0.003645,0.007140,0.012428,0.003785,0.013175,0.007034,0.051833,0.001797,0.898605,12187,383
2338672,2è,O,0.001676,0.003085,0.022823,0.003075,0.023959,0.005039,0.015503,0.002257,0.008474,0.001391,0.053641,0.005868,0.987668,1290,608
2029835,General,O,0.003433,0.007437,0.044556,0.017026,0.011985,0.005303,0.033036,0.022466,0.160172,0.003596,0.056870,0.003630,0.630493,18749,1
1163683,Experiment,O,0.002937,0.004013,0.006974,0.005103,0.025459,0.003434,0.018114,0.013690,0.009498,0.004259,0.056930,0.004340,0.845249,609,95
3780008,BP-0550,O,0.002820,0.030945,0.049136,0.009141,0.014462,0.005043,0.030631,0.045249,0.801814,0.074468,0.061845,0.006548,0.098139,11387,1604
2030561,General,O,0.005671,0.007814,0.224084,0.018871,0.016329,0.003444,0.027709,0.027130,0.255174,0.004743,0.062238,0.005813,0.340979,18749,771
3780007,-,O,0.003231,0.034222,0.062644,0.006612,0.004490,0.008848,0.010535,0.019115,0.376918,0.091159,0.063051,0.007048,0.312127,11387,1603
1183156,Cucun,O,0.009535,0.012024,0.113139,0.008577,0.087824,0.010240,0.090819,0.008161,0.057355,0.006026,0.064785,0.028613,0.688079,3709,2365
1832847,IV-8322,B-ID_NUM,0.032751,0.058076,0.036878,0.058260,0.013483,0.007757,0.040046,0.132920,0.186591,0.030172,0.064829,0.014344,0.723769,15717,365
4167108,Use,O,0.001624,0.000353,0.003004,0.000507,0.001606,0.000464,0.002786,0.000793,0.005869,0.001245,0.066763,0.001681,0.913307,16467,11


In [ ]:
df[df["B-PHONE_NUM"] > 0.1][main_cols].style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-EMAIL,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,I-USERNAME,O,document,token
195354,208798413907,B-ID_NUM,0.001319,0.924211,0.002604,0.131177,0.001019,0.000742,0.001609,0.004106,0.003589,0.001319,0.001303,0.003185,0.000628,0.000267,0.042941,9760,245
195464,943995368223,B-ID_NUM,0.000363,0.784617,0.000603,0.661486,0.000898,0.000760,0.000978,0.001460,0.001306,0.000755,0.002144,0.001688,0.000747,0.000229,0.012699,9760,401
390143,723847538279,B-ID_NUM,0.000659,0.966351,0.001984,0.142256,0.001766,0.002651,0.001824,0.003137,0.001544,0.001524,0.000777,0.003612,0.000542,0.000286,0.042789,11900,3
390428,871483046449,B-ID_NUM,0.002293,0.947403,0.010175,0.221934,0.006790,0.008312,0.003455,0.006619,0.004356,0.002639,0.001391,0.009372,0.002323,0.000758,0.200925,11900,333
390457,836172426340,B-ID_NUM,0.000970,0.957503,0.004773,0.113828,0.002943,0.005104,0.002032,0.004439,0.002149,0.003419,0.000909,0.003948,0.001332,0.000360,0.471497,11900,371
414988,369615882777,B-ID_NUM,0.001004,0.918549,0.003184,0.153547,0.001258,0.001507,0.001561,0.003441,0.002550,0.001055,0.001698,0.003022,0.000654,0.000343,0.094205,12184,6
415321,264945858442,B-ID_NUM,0.001703,0.907287,0.003876,0.236067,0.002175,0.002145,0.002613,0.006633,0.004642,0.002624,0.002245,0.006283,0.000881,0.000652,0.070851,12184,389
415351,320622779078,B-ID_NUM,0.002485,0.868235,0.005736,0.104681,0.002791,0.002627,0.002964,0.005525,0.004489,0.002412,0.003277,0.004636,0.002054,0.000400,0.209304,12184,430


In [ ]:
df[df["I-PHONE_NUM"] > 0.1][main_cols].style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-EMAIL,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,I-USERNAME,O,document,token


In [9]:
df[df.label == "B-URL_PERSONAL"][main_cols]

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,O,document,token
10166,https://youtu.be/rFD2lJuvace,B-URL_PERSONAL,0.005502,0.010760,0.005696,0.008178,0.002527,0.299088,0.001223,0.003351,0.003833,0.006986,0.004733,0.005273,0.828390,472,922
17073,http://jacobs-fisher.com/listpost.html,B-URL_PERSONAL,0.000041,0.000043,0.000103,0.000063,0.000036,0.999630,0.000013,0.000041,0.000029,0.000037,0.000030,0.000077,0.001777,2672,269
18184,https://www.youtube.com/watch?v=n-ajTPJ1h-J,B-URL_PERSONAL,0.002819,0.008077,0.004922,0.007397,0.002222,0.231158,0.001164,0.004795,0.004603,0.004543,0.003497,0.005388,0.913225,2700,825
22981,https://www.peterson.net/tag/app/listmain.php,B-URL_PERSONAL,0.000077,0.000053,0.000068,0.000086,0.000063,0.999683,0.000020,0.000041,0.000033,0.000053,0.000049,0.000053,0.001442,3592,55
77621,https://soto.com/listregister.asp,B-URL_PERSONAL,0.000053,0.000075,0.000110,0.000121,0.000042,0.999103,0.000017,0.000047,0.000036,0.000057,0.000048,0.000109,0.004460,6784,733
82970,https://ingram-castro.com/tagsmain.html,B-URL_PERSONAL,0.000057,0.000052,0.000104,0.000082,0.000053,0.999466,0.000017,0.000049,0.000034,0.000054,0.000036,0.000063,0.002223,7000,444
86129,https://www.facebook.com/amanda37,B-URL_PERSONAL,0.000039,0.000023,0.000053,0.000099,0.000020,0.999774,0.000012,0.000031,0.000020,0.000027,0.000033,0.000035,0.000309,7308,7
86483,https://www.facebook.com/amanda37,B-URL_PERSONAL,0.000057,0.000024,0.000068,0.000106,0.000022,0.999761,0.000015,0.000038,0.000024,0.000031,0.000039,0.000040,0.000457,7308,384
88084,https://jones.com/tags/wp-contentpost.html,B-URL_PERSONAL,0.000027,0.000060,0.000089,0.000091,0.000025,0.999581,0.000008,0.000032,0.000033,0.000037,0.000037,0.000082,0.001136,7408,17
93218,http://www.tate.com/,B-URL_PERSONAL,0.002340,0.000748,0.001220,0.002352,0.000237,0.977428,0.000162,0.000569,0.000707,0.000949,0.000370,0.000769,0.099098,7680,76


In [10]:
df[df["B-URL_PERSONAL"] > 0.1][main_cols].sort_values("B-URL_PERSONAL").style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,O,document,token
3534846,https://www.viddsee.com/video/bak-gia-,O,0.001315,0.000452,0.020518,0.000527,0.002150,0.101938,0.003624,0.000401,0.001754,0.000439,0.003908,0.000649,0.998467,6459,947
2420148,http://wisemapping.com/,O,0.000684,0.000386,0.000686,0.000526,0.000297,0.104374,0.000280,0.000224,0.000320,0.000370,0.000282,0.000164,0.997375,7858,464
1558048,https://www.linkedin.com/pulse/importance-visualisation-design-thinking-rizan-rizvi/,O,0.001329,0.001411,0.015192,0.002601,0.001109,0.105132,0.001342,0.002606,0.012216,0.000880,0.001010,0.002415,0.999813,11929,811
251279,www.deza.admin.ch/ressources/resource_en_155620.pdt,O,0.002717,0.002560,0.002071,0.002102,0.000473,0.113343,0.000797,0.007859,0.000549,0.000465,0.000550,0.001637,0.997911,10320,741
2245060,https://www.uservoice.com/,O,0.000168,0.000752,0.000302,0.000263,0.000639,0.114447,0.000193,0.000435,0.000290,0.000359,0.000255,0.000162,0.992160,21425,951
1183188,2182,O,0.008968,0.072386,0.004961,0.057353,0.003383,0.116046,0.004664,0.048788,0.011646,0.357521,0.012297,0.006072,0.428855,3709,2401
1183185,-,O,0.003353,0.101095,0.002693,0.041976,0.002331,0.119510,0.002521,0.010296,0.006694,0.137741,0.006202,0.005134,0.560454,3709,2398
2149061,https://www.changemakers.com/youthventure/resources/rootcause,O,0.000161,0.000641,0.000476,0.000218,0.000562,0.122487,0.000237,0.000386,0.000309,0.000303,0.000227,0.000195,0.998992,20269,206
1216134,Visualizaci%C3%B3n,O,0.009641,0.113152,0.008490,0.054097,0.011795,0.134982,0.003486,0.027234,0.014225,0.014010,0.012481,0.007350,0.999840,5433,282
1189926,http://www.greendealsolutions.net/wp-content/uploads/2013/03/strategies-for-change.jpg,O,0.001821,0.001321,0.001592,0.000766,0.000806,0.159948,0.001193,0.000746,0.000853,0.000524,0.001673,0.000457,0.999882,4381,863


In [ ]:
df[df["B-ID_NUM"] > 0.1][main_cols].sort_values("B-ID_NUM").style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-EMAIL,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,I-USERNAME,O,document,token
647946,OQ,O,0.001584,0.111609,0.012083,0.003694,0.001389,0.001304,0.001835,0.000644,0.003813,0.001733,0.000546,0.000834,0.000908,0.000193,0.960234,15452,249
763867,270220,O,0.001097,0.120009,0.023742,0.043184,0.002642,0.003114,0.003977,0.001664,0.002945,0.004735,0.001192,0.007772,0.000991,0.000433,0.819692,17076,6
97876,18:19:48,O,0.005453,0.140500,0.004436,0.032264,0.010608,0.012318,0.002486,0.002973,0.002852,0.003508,0.010024,0.012740,0.003977,0.001358,0.984711,7932,764
1035802,V69230,B-ID_NUM,0.000860,0.143468,0.011652,0.006541,0.001714,0.001478,0.010984,0.000926,0.002603,0.017188,0.002587,0.004769,0.000752,0.000334,0.960912,20984,6
333371,https://d3c33hcgiwev3.cloudfront.net/yhzsTGFxEeWsVBKLSpTUPw.processed/full/360p/index.mp4?Expires=1596153600&Sign,O,0.003024,0.230801,0.004603,0.061859,0.006822,0.811352,0.003272,0.003788,0.002920,0.002318,0.006916,0.009220,0.003942,0.000542,0.999902,11240,238
195464,943995368223,B-ID_NUM,0.000363,0.784617,0.000603,0.661486,0.000898,0.000760,0.000978,0.001460,0.001306,0.000755,0.002144,0.001688,0.000747,0.000229,0.012699,9760,401
415351,320622779078,B-ID_NUM,0.002485,0.868235,0.005736,0.104681,0.002791,0.002627,0.002964,0.005525,0.004489,0.002412,0.003277,0.004636,0.002054,0.000400,0.209304,12184,430
415321,264945858442,B-ID_NUM,0.001703,0.907287,0.003876,0.236067,0.002175,0.002145,0.002613,0.006633,0.004642,0.002624,0.002245,0.006283,0.000881,0.000652,0.070851,12184,389
414988,369615882777,B-ID_NUM,0.001004,0.918549,0.003184,0.153547,0.001258,0.001507,0.001561,0.003441,0.002550,0.001055,0.001698,0.003022,0.000654,0.000343,0.094205,12184,6
195354,208798413907,B-ID_NUM,0.001319,0.924211,0.002604,0.131177,0.001019,0.000742,0.001609,0.004106,0.003589,0.001319,0.001303,0.003185,0.000628,0.000267,0.042941,9760,245


In [ ]:
df[df["I-ID_NUM"] > 0.1][main_cols].style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-EMAIL,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,I-USERNAME,O,document,token
62398,Vw.:403489591437,B-ID_NUM,0.000097,0.999856,0.000040,0.000140,0.000057,0.000028,0.000097,0.000051,0.993913,0.000054,0.000030,0.000158,0.000043,0.000056,0.000138,5796,28
668869,.,O,0.000224,0.003661,0.000142,0.000500,0.000362,0.000557,0.000641,0.000091,0.984449,0.000124,0.000072,0.000203,0.000327,0.000143,0.008504,15720,573
668870,2000414,O,0.000156,0.000188,0.000070,0.000182,0.000084,0.000138,0.000224,0.000101,0.997797,0.000084,0.000047,0.000105,0.000099,0.000054,0.001538,15720,574
915879,30407059,I-ID_NUM,0.000453,0.014172,0.000298,0.001333,0.000414,0.000327,0.000552,0.000810,0.989434,0.000264,0.000242,0.000831,0.000342,0.000268,0.006678,19280,55


In [ ]:
df[df["B-NAME_STUDENT"] > 0.1][main_cols].sort_values(["B-NAME_STUDENT"]).style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-EMAIL,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,I-USERNAME,O,document,token
277833,M,O,0.000103,0.000498,0.100119,0.000055,0.000144,0.000343,0.000307,0.000059,0.000162,0.000207,0.000062,0.000118,0.000060,0.000046,0.897717,10556,545
734294,john,O,0.000331,0.001230,0.100281,0.000065,0.000351,0.000702,0.001821,0.000102,0.000335,0.000514,0.000079,0.000240,0.000178,0.000121,0.893650,16684,616
252528,Diane,O,0.000209,0.000601,0.100427,0.000080,0.000253,0.000204,0.000421,0.000118,0.000180,0.004895,0.000115,0.000381,0.000121,0.000072,0.891921,10324,373
243644,Maria,O,0.000186,0.000348,0.101058,0.000072,0.000173,0.000450,0.000231,0.000071,0.000193,0.000464,0.000086,0.000116,0.000082,0.000036,0.970950,10220,1215
390173,John,O,0.000162,0.000571,0.104268,0.000107,0.000184,0.000212,0.000296,0.000071,0.000170,0.000405,0.000143,0.000117,0.000072,0.000047,0.893174,11900,37
304806,Rosangela,O,0.000153,0.000202,0.107323,0.000093,0.000231,0.000444,0.000346,0.000066,0.000083,0.002117,0.000065,0.000210,0.000132,0.000080,0.974653,10908,144
75366,Jack,O,0.000051,0.000245,0.109832,0.000038,0.000160,0.000120,0.000123,0.000030,0.000120,0.000404,0.000063,0.000084,0.000030,0.000038,0.888662,6600,599
243313,Apetzuca,O,0.001392,0.001392,0.112220,0.000568,0.001142,0.002669,0.001085,0.000716,0.000590,0.011435,0.000451,0.003387,0.000934,0.000322,0.977841,10220,860
105248,Tina,O,0.000136,0.000261,0.118402,0.000053,0.000160,0.000142,0.000134,0.000036,0.000157,0.000369,0.000073,0.000114,0.000067,0.000027,0.879868,8344,1501
304939,Rosangela,O,0.000162,0.000272,0.122753,0.000101,0.000173,0.000367,0.000388,0.000054,0.000100,0.002380,0.000070,0.000168,0.000143,0.000077,0.974042,10908,287


In [12]:
df[(df["I-NAME_STUDENT"] > 0.05)&(df["I-NAME_STUDENT"] < 0.5)][main_cols].sort_values(["document", "token"]).style.applymap(lambda x: 'background-color : red' if isinstance(x, float) and x > 0.2 else '')

,token_text,label,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,O,document,token
2675,Sakir,B-NAME_STUDENT,0.000459,0.000489,0.948198,0.000278,0.000198,0.000255,0.000644,0.000553,0.101814,0.000292,0.000153,0.000214,0.002299,104,8
7569,MCMR,O,0.003055,0.000787,0.653918,0.001372,0.000892,0.001433,0.005779,0.000810,0.054371,0.000359,0.000487,0.000762,0.688944,308,1530
7571,Hunt,O,0.003260,0.001076,0.573366,0.001853,0.002305,0.002631,0.010374,0.002076,0.125147,0.001150,0.002064,0.000942,0.273756,308,1532
7583,MCMR,O,0.002840,0.001501,0.594365,0.001331,0.001000,0.001422,0.013416,0.001186,0.127160,0.000438,0.000726,0.000685,0.559402,308,1547
7585,Hunt,O,0.002799,0.001334,0.643239,0.001588,0.003140,0.002507,0.020219,0.002976,0.138997,0.001177,0.001988,0.000748,0.179288,308,1549
1163659,Kasyap,O,0.017648,0.079996,0.016917,0.027958,0.049005,0.015178,0.212365,0.045281,0.097780,0.016930,0.231506,0.026557,0.342651,609,60
1163682,Lopez,I-NAME_STUDENT,0.019049,0.008387,0.016091,0.010428,0.031092,0.016771,0.049901,0.033713,0.336366,0.065791,0.149828,0.007588,0.254995,609,93
2338682,Bello,O,0.002280,0.003003,0.005169,0.002989,0.011201,0.001620,0.017335,0.001593,0.139951,0.001649,0.028107,0.002040,0.783063,1290,622
2338698,Bello,O,0.001865,0.003024,0.010055,0.002108,0.011304,0.002052,0.007017,0.001199,0.150323,0.001182,0.002584,0.001112,0.806175,1290,641
2342237,Sharma,I-NAME_STUDENT,0.003477,0.003081,0.036554,0.003731,0.003785,0.003840,0.011735,0.001211,0.281473,0.002024,0.003681,0.006553,0.638855,1798,71


In [ ]:
pred_cols = ['B-EMAIL', 'B-ID_NUM',
       'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL',
       'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM',
       'I-STREET_ADDRESS', 'I-URL_PERSONAL', 'O',]


for p in pred_cols:
    if p == "O": continue

    ll = p.split("-")[-1]

    temp = df[df.label == p]

    if len(temp) == 0:
        continue

    argmax_col = [pred_cols[x] for x in temp[pred_cols].values.argmax(-1)]

    argmax_acc = sum([x == p for x in argmax_col]) / len(argmax_col)

    print(p, len(temp), round(argmax_acc, 3), round(temp[p].mean(), 3), round(temp[p].max(), 3), round(temp[p].min(), 3), sep="\t\t\t\t")

    for i in range(100):
        i /= 100




B-EMAIL				39				1.0				0.998				1.0				0.992
B-ID_NUM				78				0.962				0.949				1.0				0.0
B-NAME_STUDENT				1365				0.918				0.901				0.999				0.0
B-PHONE_NUM				6				1.0				0.993				0.998				0.98
B-STREET_ADDRESS				2				0.5				0.601				0.866				0.337
B-URL_PERSONAL				110				0.945				0.949				1.0				0.231
B-USERNAME				6				0.833				0.905				0.997				0.655
I-ID_NUM				1				1.0				0.986				0.986				0.986
I-NAME_STUDENT				1096				0.981				0.971				0.999				0.0
I-PHONE_NUM				15				1.0				0.999				0.999				0.999
I-STREET_ADDRESS				18				1.0				0.991				0.996				0.984
I-URL_PERSONAL				1				0.0				0.011				0.011				0.011


In [ ]:
df["doc-tok"] = df["document"].astype(str) +"_" +  df["token"].astype(str)
temp = df[(df["B-NAME_STUDENT"] > 0.05) | (df["I-NAME_STUDENT"] > 0.05)].copy()



name_doc_tok = df[df.label.str.contains("NAME_STUDENT")]["doc-tok"]
temp_name_doc_tok = temp[temp.label.str.contains("NAME_STUDENT")]["doc-tok"]

len(name_doc_tok), len(set(name_doc_tok)-set(temp_name_doc_tok))

(2461, 34)

In [ ]:
set(name_doc_tok)-set(temp_name_doc_tok)

{'11566_966',
 '12330_125',
 '12330_163',
 '12330_2',
 '12330_221',
 '12330_337',
 '12330_390',
 '12512_787',
 '12512_792',
 '12585_3',
 '12585_4',
 '204_4',
 '2722_809',
 '2722_952',
 '3241_632',
 '4094_80',
 '4094_81',
 '4161_286',
 '4922_3',
 '4922_4',
 '6117_62',
 '6117_63',
 '9050_314',
 '9281_796',
 '9281_797',
 '9289_613',
 '9365_401',
 '9674_730',
 '9674_735',
 '9674_741',
 '9760_2',
 '9760_220',
 '9760_221',
 '9760_3'}

In [ ]:
temp[temp.token_text.str.istitle().copy()]

,token,label,document,token_text,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,O,row_id,doc-tok
3,4,B-NAME_STUDENT,16,Gilberto,0.000174,0.000297,0.992107,0.000200,0.000047,0.000181,0.000246,0.000207,0.001565,0.000134,0.000065,0.000079,0.004698,3,16_4
4,5,I-NAME_STUDENT,16,Gamboa,0.000070,0.000186,0.000328,0.000113,0.000081,0.000045,0.000272,0.000107,0.997967,0.000054,0.000169,0.000084,0.000896,4,16_5
681,5,B-NAME_STUDENT,20,Sindy,0.000062,0.000113,0.997962,0.000052,0.000028,0.000063,0.000090,0.000055,0.000524,0.000042,0.000021,0.000030,0.001943,681,20_5
682,6,I-NAME_STUDENT,20,Samaca,0.000040,0.000208,0.000934,0.000080,0.000058,0.000045,0.000127,0.000074,0.997907,0.000050,0.000076,0.000092,0.001162,682,20_6
976,328,O,20,George,0.000190,0.000154,0.945900,0.000153,0.000062,0.000177,0.000230,0.000175,0.000509,0.000172,0.000054,0.000113,0.052112,976,20_328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4523393,283,O,21711,Pierre,0.000493,0.001246,0.767066,0.001054,0.000611,0.001792,0.000539,0.000195,0.004344,0.000953,0.000414,0.000231,0.221063,1053007,21711_283
4550648,10,O,22023,Vinay,0.000052,0.000070,0.998966,0.000051,0.000043,0.000021,0.000024,0.000023,0.000434,0.000036,0.000052,0.000049,0.000179,1080262,22023_10
4550649,11,O,22023,Kolusu,0.000041,0.000017,0.000110,0.000023,0.000044,0.000034,0.000104,0.000050,0.999212,0.000056,0.000051,0.000074,0.000286,1080263,22023_11
4551156,561,O,22023,Vinay,0.000044,0.000052,0.999041,0.000051,0.000040,0.000020,0.000020,0.000017,0.000358,0.000028,0.000040,0.000038,0.000249,1080770,22023_561


In [ ]:
temp[temp.token_text.str.istitle()]["label"].value_counts()

B-NAME_STUDENT    1340
O                 1192
I-NAME_STUDENT    1087
B-ID_NUM             1
Name: label, dtype: int64

In [ ]:
temp[temp.token_text.str.istitle()][["token", "document", "label"]].to_parquet("fp_names.pq")